<a href="https://colab.research.google.com/github/serereuk/TensorFlow-Examples/blob/master/tensorflow_batch_handling_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import cifar100
import tensorflow as tf
import numpy as np

Using TensorFlow backend.


In [0]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

In [0]:
writer = tf.python_io.TFRecordWriter('train.tfrecords')
for index in range(x_train.shape[0]):
  image = x_train[index].tostring()
  example = tf.train.Example(features = tf.train.Features(feature = {
      'image_raw' : tf.train.Feature(bytes_list = tf.train.BytesList(value = [image])),
      'label' : tf.train.Feature(int64_list= tf.train.Int64List(value= y_train[index]))
  }))
  writer.write(example.SerializeToString())
writer.close()

In [0]:
writer = tf.python_io.TFRecordWriter('test.tfrecords')
for index in range(x_test.shape[0]):
  image = x_test[index].tostring()
  example = tf.train.Example(features = tf.train.Features(feature = {
      'image_raw' : tf.train.Feature(bytes_list = tf.train.BytesList(value = [image])),
      'label' : tf.train.Feature(int64_list= tf.train.Int64List(value= y_test[index]))
  }))
  writer.write(example.SerializeToString())
writer.close()

In [5]:
filename_queue = tf.train.string_input_producer(['train.tfrecords'], num_epochs = 10)

W0714 14:14:18.038146 140127785719680 deprecation.py:323] From <ipython-input-5-96b32c86e22e>:1: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0714 14:14:18.073935 140127785719680 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/input.py:278: input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0714 

In [6]:
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(
  serialized_example,
  features = {
      'image_raw': tf.FixedLenFeature([], tf.string),
      'label' : tf.FixedLenFeature([], tf.int64)
  }
)

W0714 14:14:20.759721 140127785719680 deprecation.py:323] From <ipython-input-6-4ebb6a7adbfb>:1: TFRecordReader.__init__ (from tensorflow.python.ops.io_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecordDataset`.


In [7]:
image = tf.decode_raw(features['image_raw'], tf.uint8)
image.set_shape([32*32*3])
image = tf.cast(image, tf.float32) * (1./255) - 0.5
label = tf.cast(features['label'], tf.int32)
images_batch, labels_batch = tf.train.shuffle_batch([image, label], batch_size=128,
                                                   capacity=2000, min_after_dequeue = 1000)

W0714 14:14:23.072533 140127785719680 deprecation.py:323] From <ipython-input-7-167a1212be2d>:6: shuffle_batch (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.shuffle(min_after_dequeue).batch(batch_size)`.


In [8]:
W = tf.get_variable('W', [32*32*3, 100])
y_pred = tf.matmul(images_batch, W)
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = y_pred, labels=labels_batch)
loss_mean = tf.reduce_mean(loss)
train_op = tf.train.AdamOptimizer().minimize(loss)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
init = tf.local_variables_initializer()
sess.run(init)
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord = coord)

try:
  step = 0
  while not coord.should_stop():
    step += 1
    sess.run([train_op])
    if step % 500 == 0:
      loss_mean_eval = sess.run([loss_mean])
      print(step); print(loss_mean_eval)
except tf.errors.OutOfRangeError:
  print("Done")
finally:
  coord.request_stop()
coord.join(threads)
sess.close

W0714 14:14:25.273025 140127785719680 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0714 14:14:28.277313 140127785719680 deprecation.py:323] From <ipython-input-8-4f034eed94b9>:12: start_queue_runners (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


500
[3.6969924]
1000
[3.7616475]


KeyboardInterrupt: ignored